In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns
from statannotations.Annotator import Annotator

# enables concurrent editing of base.py
from importlib import reload
reload(base)

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)

In [ ]:
# Setup data loading
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'/'2024.12.04_exp092.3'/'export'

plates = pd.DataFrame({
    'data_path': [base_path],
    'yaml_path': [base_path/'wells.yaml'],
    'biorep': [4],
    'exp': ['exp092.3',]
})
output_path = rd.rootdir/'output'/'ts_num'
cache_path = output_path/'data.gzip'

for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)

In [ ]:
# Load data
data = pd.DataFrame()
channel_list = ['mRuby2-A','mGL-A']

if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)
    for c in channel_list: data = data[data[c]>0]
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Draw gates
gates = pd.DataFrame()
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

display(gates)

In [ ]:
plot_df = data.groupby(['exp','construct']).sample(1000)
g = sns.displot(data=plot_df, x='mGL-A', y='mRuby2-A', hue='exp', 
                kind='kde', col='construct', col_wrap=5,
                common_norm=False, levels=5, log_scale=True)

for _, ax in g.axes_dict.items():
    ax.axvline(gates['mGL-A'].values[0], c='black', ls=':', zorder=0)
    ax.axhline(gates['mRuby2-A'].values[0], c='black', ls=':', zorder=0)

In [ ]:
# Gate data by marker expression
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
data['gated'] = data['expressing'] & (data['construct']!='UT')

In [ ]:
# Bin data and calculate statistics
df_quantiles, df_stats = base.calculate_bins_stats(data[data['gated']])

# Add metadata
metadata = base.get_metadata(metadata_path/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
df_quantiles = df_quantiles.merge(metadata, how='left', on='construct')
df_stats = df_stats.merge(metadata, how='left', on='construct')